In [1]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### List of albums I am interested on

In [2]:
topalbums = pd.read_csv('../data/clean/DB_Kaggle_top500albums_clean.csv', encoding='latin1')

topalbums.head(3)

,toprankingorder,album,artist,year
0,1,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1967
1,2,Pet Sounds,The Beach Boys,1966
2,3,Revolver,The Beatles,1966


In [3]:
horriblealbums = pd.read_csv('../data/clean/WS_50horriblealbums_clean.csv', encoding='latin1')

horriblealbums.head(3)

,horriblerankingorder,album,artist,rationale,year
0,50,Its Hard,The Who,"In the early Eighties, Pete Townshend was jugg...",1982
1,49,The Bridge,Billy Joel,Billy Joel had nearly a solid decade of succes...,1986
2,48,Van Halen III,Van Halen,When original Van Halen singer David Lee Roth ...,1998


In [4]:
#need to be conscious of the fact that some albums have the same name 

topalbums['album'].value_counts().head(3)

album
Greatest Hits                            3
Let It Be                                2
Sgt. Pepper's Lonely Hearts Club Band    1
Name: count, dtype: int64

In [5]:
#let us try to get unique values for album_artist 

topalbums['album_artist'] = topalbums['album'] + ' - ' + topalbums['artist'] 

In [6]:
topalbums.head(3)

,toprankingorder,album,artist,year,album_artist
0,1,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1967,Sgt. Pepper's Lonely Hearts Club Band - The Be...
1,2,Pet Sounds,The Beach Boys,1966,Pet Sounds - The Beach Boys
2,3,Revolver,The Beatles,1966,Revolver - The Beatles


In [7]:
horriblealbums['album'].value_counts().head(2)

album
Its Hard           1
Leather Jackets    1
Name: count, dtype: int64

In [8]:
horriblealbums['album_artist'] = horriblealbums['album'] + ' - ' + horriblealbums['artist']

In [9]:
horriblealbums.head(2)

,horriblerankingorder,album,artist,rationale,year,album_artist
0,50,Its Hard,The Who,"In the early Eighties, Pete Townshend was jugg...",1982,Its Hard - The Who
1,49,The Bridge,Billy Joel,Billy Joel had nearly a solid decade of succes...,1986,The Bridge - Billy Joel


In [10]:
all_albums = pd.concat([topalbums, horriblealbums], ignore_index=True)
all_albums.shape

(550, 7)

In [11]:
all_albums.head(550)

,toprankingorder,album,artist,year,album_artist,horriblerankingorder,rationale
0,1.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1967,Sgt. Pepper's Lonely Hearts Club Band - The Be...,NaN,NaN
1,2.0,Pet Sounds,The Beach Boys,1966,Pet Sounds - The Beach Boys,NaN,NaN
2,3.0,Revolver,The Beatles,1966,Revolver - The Beatles,NaN,NaN
3,4.0,Highway 61 Revisited,Bob Dylan,1965,Highway 61 Revisited - Bob Dylan,NaN,NaN
4,5.0,Rubber Soul,The Beatles,1965,Rubber Soul - The Beatles,NaN,NaN
...,...,...,...,...,...,...,...
545,NaN,Down in the Groove,Bob Dylan,1988,Down in the Groove - Bob Dylan,5.0,Dylan aficionados have been arguing for decade...
546,NaN,Metal Magic,Pantera,1983,Metal Magic - Pantera,4.0,Pantera are undoubtedly one of the greatest me...
547,NaN,Union,Yes,1991,Union - Yes,3.0,"By the late Eighties, prog-rockers Yes had spl..."
548,NaN,Squeeze,The Velvet Underground,1973,Squeeze - The Velvet Underground,2.0,The Velvet Underground were a band in name onl...


In [12]:
album_artist = all_albums['album_artist'].unique().tolist()

In [13]:
len(album_artist)

550

### Spotify Connection 

In [14]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [72]:
import requests as rq
import pandas as pd
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import time
import pymongo
from passwords import *

In [73]:
Client_ID

'eda83f2b172041569cc4822a12a4c8ab'

In [74]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Album Information 

In [75]:
albums = pd.DataFrame(all_albums['album'])
albums.head(4)

,album
0,Sgt. Pepper's Lonely Hearts Club Band
1,Pet Sounds
2,Revolver
3,Highway 61 Revisited


In [76]:
#test 

In [77]:
url = 'https://api.spotify.com/v1/search?q=album%3Apet+sounds&type=album&limit=50&offset=0'

In [78]:
res = rq.get(url, headers=headers)
res.status_code 

200

In [79]:
data = res.json()

In [80]:
type(data)

dict

In [81]:
data.keys()

dict_keys(['albums'])

In [82]:
data['albums'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [83]:
data['albums']['items'][0].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [84]:
#quiero el id del album en Spotify 
data['albums']['items'][0]['id']

'6GphKx2QAPRoVGWE9D7ou8'

In [85]:
#quiero el nombre del album 
data['albums']['items'][0]['name']

'Pet Sounds (Original Mono & Stereo Mix)'

In [86]:
#quiero el nombre del artista del album 
data['albums']['items'][0]['artists'][0]['name']

'The Beach Boys'

In [87]:
#quiero el id del artista del album 
data['albums']['items'][0]['artists'][0]['id']

'3oDbviiivRWhXwIE8hxkVV'

In [88]:
#quiero el popularity del album 
#data['albums']['items'][0]['popularity']

In [89]:
#quiero el release_date del album 
data['albums']['items'][0]['release_date']

'1966-06-16'

In [90]:
#quiero el total de tracks del album 
data['albums']['items'][0]['total_tracks']

27

In [91]:
# Replace spaces with the '+' character in the album names
consulta = [x.replace(' ', '+') for x in albums.album]

# Construct a list of URLs for querying album information
album_urls = [f'https://api.spotify.com/v1/search?q=album%3A{album}&type=album&limit=50&offset=0' for album in consulta]

# Create a list of dictionaries with album information and corresponding URLs
album_url_list = [{'album': album, 'url': url} for album, url in zip(albums.album, album_urls)]

In [95]:
# List to store the retrieved album data
album_data_list = []

# Function to search and retrieve album information
def search_spotify_info(album_url):
    time.sleep(0.20)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(album_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()
        rjson = data.get('albums', {}).get('items', [])

        for index, item in enumerate(rjson):
            if item['name'].lower() == album_url['album'].lower():
                album_data_list.append({
                    'ID': item['id'],  # Spotify ID for the album
                    'Name': item['name'],
                    'Artist': item['artists'][0]['name'],
                    #'Popularity': item['popularity'],
                    'Release Date': item['release_date'],
                    'Total Tracks': item['total_tracks']
                })
                
                return rjson[index]

        # If the album is not found, raise an exception to return None
        raise Exception("Album not found")
    except Exception as e:
        print(f"An error occurred: {e}")
        album_data_list.append({
            'ID': None,
            'Name': None,
            'Artist': None,
            'Popularity': None,
            'Release Date': None,
            'Total Tracks': None
        })
        return {
            'album_type': None,
            'artists': None,
            'available_markets': None,
            'external_urls': None,
            'href': None,
            'id': None,
            'images': None,
            'name': None,
            'release_date': None,
            'release_date_precision': None,
            'total_tracks': None,
            'type': None,
            'uri': None
        }
    
# Execute the function for each album URL
for album_url in album_url_list:
    search_spotify_info(album_url)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(album_data_list)

An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: A

In [96]:
df.head()

,ID,Name,Artist,Release Date,Total Tracks,Popularity
0,1x1jpjDbetGqX0IKCUIBNj,Sgt. Pepper's Lonely Hearts Club Band,Bloco do Sargento Pimenta,2017-12-08,13.0,NaN
1,2CNEkSE8TADXRT2AzcEt1b,Pet Sounds,The Beach Boys,1966-05-16,13.0,NaN
2,0T1sskJDoybYGvPU5aw5Cf,REVOLVER,Lil Darkie,2020-06-15,1.0,NaN
3,6YabPKtZAjxwyWbuO9p4ZD,Highway 61 Revisited,Bob Dylan,1965-08-30,9.0,NaN
4,5TIQEIzrI6RQfUVQ5Y571D,Rubber Soul,The Beatles Complete On Ukulele,2020-07-28,16.0,NaN
